In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import roc_auc_score
import matplotlib as plt
%matplotlib inline
from mypipes import *
import sklearn

sklearn.__version__

'1.0.2'

In [2]:
train_file=r'\Users\91878\Downloads\titanic\train.csv'
test_file=r'\Users\91878\Downloads\titanic\test.csv'

tit_train=pd.read_csv(train_file)
tit_test=pd.read_csv(test_file)

In [3]:
tit_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
tit_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
tit_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
tit_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
num_vars=list(tit_train.select_dtypes(exclude=['object']).columns)

In [8]:
num_vars


['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [9]:
num_vars=[_ for _ in num_vars if _ not in ['Survived']]
num_vars

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [10]:
cat_vars=list(tit_train.select_dtypes(include=['object']).columns)
cat_vars

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [11]:
cat_vars=[_ for _ in cat_vars if _ not in ['Ticket','Name']]
cat_vars

['Sex', 'Cabin', 'Embarked']

In [12]:
p1=pdPipeline([
    ('var_select',VarSelector(num_vars)),
    ('missing_trt',DataFrameImputer())
])

p2=pdPipeline([
    ('var_select',VarSelector(cat_vars)),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(70))
])

In [13]:
data_pipe=FeatureUnion([
    ('num',p1),
    ('obj_to_dum',p2),
    
])

In [14]:
x_train=pd.DataFrame(data=data_pipe.fit_transform(tit_train),
                     columns=data_pipe.get_feature_names())


C:\Users\91878\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
x_test=pd.DataFrame(data=data_pipe.transform(tit_test),
                     columns=data_pipe.get_feature_names())

In [16]:
tit_train['Survived'].value_counts(dropna=False)

0    549
1    342
Name: Survived, dtype: int64

In [17]:
y_train=(tit_train['Survived']==1).astype(int)

In [18]:
x_train.shape

(891, 11)

In [19]:
x_test.shape

(418, 11)

In [20]:
x_train.head()

,num__PassengerId,num__Pclass,num__Age,num__SibSp,num__Parch,num__Fare,obj_to_dum__Sex_male,obj_to_dum__Cabin_missing,obj_to_dum__Embarked_S,obj_to_dum__Embarked_C,obj_to_dum__Embarked_Q
0,1.0,3.0,22.0,1.0,0.0,7.2500,1.0,1.0,1.0,0.0,0.0
1,2.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,1.0,0.0
2,3.0,3.0,26.0,0.0,0.0,7.9250,0.0,1.0,1.0,0.0,0.0
3,4.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0,1.0,0.0,0.0
4,5.0,3.0,35.0,0.0,0.0,8.0500,1.0,1.0,1.0,0.0,0.0


In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
params={'class_weight':['balanced',None],
        'solver': ['lbfgs', 'liblinear'],
        'penalty':['l1','l2'],
        'C':[.0001,.0005,.001,.005,.01,.05,.1,1,2,5]}

In [23]:
model=LogisticRegression(fit_intercept=True)

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring="roc_auc",
                         n_jobs=-1,
                         verbose=20)

In [26]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


C:\Users\91878\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\91878\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91878\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\91878\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
V

GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1,
                               2, 5],
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'liblinear']},
             scoring='roc_auc', verbose=20)

In [27]:
grid_search.best_estimator_

LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [28]:
logr=grid_search.best_estimator_

In [29]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [30]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.85430 (std: 0.02465)
Parameters: {'C': 1, 'class_weight': None, 'penalty': 'l1', 'solver': 'liblinear'}

Model with rank: 2
Mean validation score: 0.85430 (std: 0.02492)
Parameters: {'C': 1, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}

Model with rank: 3
Mean validation score: 0.85420 (std: 0.02485)
Parameters: {'C': 2, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}

Model with rank: 4
Mean validation score: 0.85408 (std: 0.02370)
Parameters: {'C': 1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

Model with rank: 5
Mean validation score: 0.85398 (std: 0.02530)
Parameters: {'C': 5, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}



In [33]:
logr=LogisticRegression(fit_intercept=True,
                        **{'C': 0.005, 'class_weight': 'balanced', 'penalty': 'l1'},solver='liblinear')

In [36]:
logr.fit(x_train,y_train)

LogisticRegression(C=0.005, class_weight='balanced', penalty='l1',
                   solver='liblinear')

In [45]:
(logr.coef_[0]==0).sum()

8

In [46]:
list(zip(x_train.columns,logr.coef_[0]))

[('num__PassengerId', -0.00016826196062819728),
 ('num__Pclass', 0.0),
 ('num__Age', -0.013639364133301432),
 ('num__SibSp', 0.0),
 ('num__Parch', 0.0),
 ('num__Fare', 0.015718895012453812),
 ('obj_to_dum__Sex_male', 0.0),
 ('obj_to_dum__Cabin_missing', 0.0),
 ('obj_to_dum__Embarked_S', 0.0),
 ('obj_to_dum__Embarked_C', 0.0),
 ('obj_to_dum__Embarked_Q', 0.0)]

In [47]:
logr.predict_proba(x_test)

array([[6.21890773e-01, 3.78109227e-01],
       [6.64023379e-01, 3.35976621e-01],
       [6.99262827e-01, 3.00737173e-01],
       [5.94522303e-01, 4.05477697e-01],
       [5.64066589e-01, 4.35933411e-01],
       [5.49060455e-01, 4.50939545e-01],
       [6.08344229e-01, 3.91655771e-01],
       [5.12508143e-01, 4.87491857e-01],
       [5.70357351e-01, 4.29642649e-01],
       [5.14600687e-01, 4.85399313e-01],
       [6.00982666e-01, 3.99017334e-01],
       [5.91618455e-01, 4.08381545e-01],
       [3.04210467e-01, 6.95789533e-01],
       [6.46311669e-01, 3.53688331e-01],
       [4.58071708e-01, 5.41928292e-01],
       [5.11052697e-01, 4.88947303e-01],
       [6.07312952e-01, 3.92687048e-01],
       [5.80738990e-01, 4.19261010e-01],
       [5.97920741e-01, 4.02079259e-01],
       [6.57796273e-01, 3.42203727e-01],
       [4.92479960e-01, 5.07520040e-01],
       [5.56393032e-01, 4.43606968e-01],
       [5.09415677e-01, 4.90584323e-01],
       [3.71817670e-01, 6.28182330e-01],
       [3.504570

In [49]:
logr.classes_

array([0, 1])

In [50]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [51]:
logr.predict_proba(x_train)

array([[0.54643389, 0.45356611],
       [0.35391773, 0.64608227],
       [0.55738503, 0.44261497],
       ...,
       [0.54062996, 0.45937004],
       [0.5082017 , 0.4917983 ],
       [0.61409529, 0.38590471]])

In [52]:
train_score=logr.predict_proba(x_train)[:,1]
real=y_train

In [53]:
(train_score>0.2).astype(int)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [54]:
 KS=[]

In [55]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
    
      
    KS=(TP/P)-(FP/N)
       
    KS_all.append(KS)
    

In [56]:
list(zip(cutoffs,KS_all))

[(0.01, 0.0),
 (0.02, 0.0),
 (0.03, 0.0),
 (0.04, 0.0),
 (0.05, 0.0),
 (0.060000000000000005, 0.0),
 (0.06999999999999999, 0.0),
 (0.08, 0.0),
 (0.09, 0.0),
 (0.09999999999999999, 0.0),
 (0.11, 0.0),
 (0.12, 0.0),
 (0.13, 0.0),
 (0.14, 0.0),
 (0.15000000000000002, 0.0),
 (0.16, 0.0),
 (0.17, 0.0),
 (0.18000000000000002, 0.0),
 (0.19, 0.0),
 (0.2, 0.0),
 (0.21000000000000002, 0.0),
 (0.22, 0.0),
 (0.23, 0.0),
 (0.24000000000000002, 0.0),
 (0.25, 0.0),
 (0.26, 0.0),
 (0.27, 0.0018214936247723523),
 (0.28, 0.0018214936247723523),
 (0.29000000000000004, 0.0036429872495445936),
 (0.3, 0.005464480874316946),
 (0.31, 0.007285974499089298),
 (0.32, 0.005081008532259568),
 (0.33, 0.005800019173617055),
 (0.34, 0.013085993672706353),
 (0.35000000000000003, 0.03130092992042943),
 (0.36000000000000004, 0.03820343207746146),
 (0.37, 0.04874892148403798),
 (0.38, 0.06801840667241876),
 (0.39, 0.10152430255967793),
 (0.4, 0.12587479628031828),
 (0.41000000000000003, 0.160722845364778),
 (0.4200000000

In [57]:
mycutoff=cutoffs[KS_all==max(KS_all)]
mycutoff

array([0.46])

In [58]:
logr.intercept_

array([0.])

In [59]:
list(zip(x_train.columns,logr.coef_[0]))

[('num__PassengerId', -0.00016826196062819728),
 ('num__Pclass', 0.0),
 ('num__Age', -0.013639364133301432),
 ('num__SibSp', 0.0),
 ('num__Parch', 0.0),
 ('num__Fare', 0.015718895012453812),
 ('obj_to_dum__Sex_male', 0.0),
 ('obj_to_dum__Cabin_missing', 0.0),
 ('obj_to_dum__Embarked_S', 0.0),
 ('obj_to_dum__Embarked_C', 0.0),
 ('obj_to_dum__Embarked_Q', 0.0)]

In [60]:
logr.predict_proba(x_test)

array([[6.21890773e-01, 3.78109227e-01],
       [6.64023379e-01, 3.35976621e-01],
       [6.99262827e-01, 3.00737173e-01],
       [5.94522303e-01, 4.05477697e-01],
       [5.64066589e-01, 4.35933411e-01],
       [5.49060455e-01, 4.50939545e-01],
       [6.08344229e-01, 3.91655771e-01],
       [5.12508143e-01, 4.87491857e-01],
       [5.70357351e-01, 4.29642649e-01],
       [5.14600687e-01, 4.85399313e-01],
       [6.00982666e-01, 3.99017334e-01],
       [5.91618455e-01, 4.08381545e-01],
       [3.04210467e-01, 6.95789533e-01],
       [6.46311669e-01, 3.53688331e-01],
       [4.58071708e-01, 5.41928292e-01],
       [5.11052697e-01, 4.88947303e-01],
       [6.07312952e-01, 3.92687048e-01],
       [5.80738990e-01, 4.19261010e-01],
       [5.97920741e-01, 4.02079259e-01],
       [6.57796273e-01, 3.42203727e-01],
       [4.92479960e-01, 5.07520040e-01],
       [5.56393032e-01, 4.43606968e-01],
       [5.09415677e-01, 4.90584323e-01],
       [3.71817670e-01, 6.28182330e-01],
       [3.504570

In [61]:
test_score=logr.predict_proba(x_test)[:,1]
test_score

array([0.37810923, 0.33597662, 0.30073717, 0.4054777 , 0.43593341,
       0.45093954, 0.39165577, 0.48749186, 0.42964265, 0.48539931,
       0.39901733, 0.40838154, 0.69578953, 0.35368833, 0.54192829,
       0.4889473 , 0.39268705, 0.41926101, 0.40207926, 0.34220373,
       0.50752004, 0.44360697, 0.49058432, 0.62818233, 0.96495429,
       0.35243786, 0.6270809 , 0.41387977, 0.44162032, 0.4511319 ,
       0.39450501, 0.50312363, 0.42992509, 0.45786605, 0.58486437,
       0.42686046, 0.39854958, 0.42395045, 0.41383397, 0.58649571,
       0.38275766, 0.46964888, 0.35595202, 0.41043674, 0.51370391,
       0.40755713, 0.42439674, 0.39697658, 0.55544434, 0.40145141,
       0.61237019, 0.42785037, 0.48248417, 0.97322534, 0.42648908,
       0.54043031, 0.3745428 , 0.40602677, 0.42832783, 0.96991585,
       0.43341257, 0.40501593, 0.42942218, 0.41597279, 0.9778187 ,
       0.44699192, 0.42975491, 0.46606918, 0.4661086 , 0.95906635,
       0.40917152, 0.41961607, 0.3933687 , 0.47119335, 0.93611

In [62]:
(test_score>mycutoff).astype(int)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [144]:
pd.DataFrame(test_score).to_csv("mytitanic.csv",index=False)

In [145]:
test_classes=(test_score>mycutoff).astype(int)

In [146]:
pd.DataFrame(test_classes).to_csv("mysubmission.csv",index=False)

In [64]:
# Assuming 'final_predictions_df' is your DataFrame with 'passenger_id' and binary predictions
passenger_id_column = pd.read_csv['passenger_id']

# Hard classification based on the threshold 'mycutoff'
test_classes = (final_predictions_df['test_score'] > mycutoff).astype(int)

# Create a DataFrame with 'passenger_id' and 'survived' columns
submission_df = pd.DataFrame({'passenger_id': passenger_id_column, 'survived': test_classes})

# Save the DataFrame to a CSV file
submission_df.to_csv("mysubmission.csv", index=False)


TypeError: 'function' object is not subscriptable